In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np


import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import time
import json
import pickle
from preprocessing_dailydialogue import *

In [2]:
# Define constant
# Default word tokens
#
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
MAX_LENGTH = 25  # Maximum sentence length to consider
MIN_COUNT = 1    # Minimum word count threshold for trimming
save_dir = os.path.join("data", "save")


In [4]:
with open('processed_train.pickle','rb') as f:
    pairs = pickle.load(f)
    pairs_emotion = pickle.load(f)
    voc = pickle.load(f)

In [11]:
for each in pairs:
    voc.addSentence(each[0])
    voc.addSentence(each[1])

In [5]:
emotion_count = {}
non_emotion = 0
have_emotion = 0
for conversation,emotions in zip(pairs,pairs_emotion):
    post,response = conversation
    post_e,response_e = emotions
    
    if post_e != 0:
        have_emotion += 1
        for word in post.split(' '):
            emotion_count[word] = emotion_count.get(word,0) + 1
    else:
        non_emotion += 1
    if response_e != 0:
        have_emotion += 1
        for word in response.split(' '):
            emotion_count[word] = emotion_count.get(word,0) + 1
    else:
        non_emotion += 1

print('No emotion output {}'.format(non_emotion))
print('Emotion output {}'.format(have_emotion))


No emotion output 142948
Emotion output 285774


In [14]:
for each in emotion_count:
    try:
        emotion_count[each] = emotion_count[each] / voc.word2count[each]
    except KeyError:
        continue

In [35]:
ememory = []
threshold = 0.60
min_count_emotion = 15


In [36]:
for each in emotion_count:
    try:
        if voc.word2count[each] > min_count_emotion and emotion_count[each] > threshold:
            ememory.append(each)
    except KeyError:
        continue
print('Total {} words'.format(len(ememory)))
        

Total 4427 words


In [37]:
ememory

['there',
 'was',
 '.',
 'you',
 'tell',
 'me',
 'what',
 'happened',
 '?',
 'it',
 's',
 'mark',
 'the',
 'so',
 'called',
 'art',
 'of',
 'acting',
 'not',
 'i',
 'a',
 'particularly',
 'hear',
 'know',
 'somewhere',
 'known',
 'my',
 'father',
 'william',
 'starks',
 'look',
 're',
 'almost',
 'us',
 'now',
 'michael',
 'we',
 'this',
 'as',
 'after',
 'given',
 'been',
 '.yes',
 '!',
 'don',
 't',
 'think',
 'why',
 'girls',
 'world',
 'dope',
 'majesty',
 'had',
 'women',
 'that',
 'desire',
 'concern',
 'no',
 'drop',
 'couple',
 'dead',
 'get',
 'out',
 'gone',
 'didn',
 'to',
 'pee',
 'then',
 'threat',
 'mentioned',
 'only',
 'passing',
 'cool',
 'off',
 'nice',
 'isn',
 'let',
 'talk',
 'about',
 'mean',
 'couldn',
 'snake',
 'shoot',
 'cuervo',
 'made',
 'swear',
 'god',
 'man',
 'alike',
 'still',
 'if',
 'say',
 'ya',
 'nothin',
 'here',
 'means',
 'said',
 'maybe',
 'll',
 'again',
 'those',
 'doctors',
 'real',
 'miracle',
 'he',
 'escaped',
 'they',
 'were',
 'silver',


In [15]:
len(ememory)

2136

In [38]:
with open('ememory.txt','w') as f:
    for each in ememory:
        f.write(each +'\n')
    